In [1]:
# main.py
import boto3
from decouple import config
import sqlite3

# Load S3-related configurations from the .env file
aws_access_key_id = config('AWS_ACCESS_KEY_ID')
aws_secret_access_key = config('AWS_SECRET_ACCESS_KEY')
bucket_name = config('AWS_BUCKET_NAME')
bucket_region = config('AWS_DEFAULT_REGION')
database_name = config('DATABASE_NAME')

UndefinedValueError: AWS_BUCKET_REGION not found. Declare it as envvar or define a default value.

In [ ]:



def main():


    # Initialize the S3 client
    s3 = boto3.client('s3',
                      aws_access_key_id=aws_access_key_id,
                      aws_secret_access_key=aws_secret_access_key,
                      region_name=bucket_region)

    # Connect to the SQLite database
    conn = sqlite3.connect(database_name)
    c = conn.cursor()

    # List objects (files) in the specified S3 folder with the given pattern
    objects = s3.list_objects_v2(Bucket=bucket_name, Prefix='data/audio/dh-new_scapes')

    if 'Contents' in objects:
        for obj in objects['Contents']:
            key = obj['Key']
            if key.endswith(".wav"):
                print(f"Reading file: {key}")

                # Read the audio file content directly from S3 using smart_open
                with open(f's3://{bucket_name}/{key}', 'rb') as file:
            
                    # Now you can insert the S3 link and other relevant data into your SQLite database
                    # For example, insert it into a table named 'audio_files'
                    c.execute("INSERT INTO audio_files (s3_link, file_name) VALUES (?, ?)", (f's3://{bucket_name}/{key}', key))

    # Commit the changes and close the database connection
    conn.commit()
    conn.close()




In [5]:
if __name__ == "__main__":
    main()

UndefinedValueError: AWS_ACCESS_KEY_ID not found. Declare it as envvar or define a default value.